### 임상도 데이터 전처리

In [2]:
import geopandas as gpd

In [ ]:
gdf = gpd.read_file("../data/raw/임상도_서울특별시/11.shp")

In [ ]:
if gdf.crs != "epsg:5179":
    gdf = gdf.to_crs(epsg=5179)
    print("Coordinate Reference System Converted")

In [ ]:
gdf = gdf.rename(columns={"FRTP_CD": "수종코드"})

In [ ]:
frtp_map = {'1': '침엽수', '2': '활엽수', '3': '혼효림'}
gdf["수종"] = gdf["수종코드"].map(frtp_map).fillna("기타")

In [ ]:
gdf["면적"] = gdf.geometry.area

In [ ]:
drop_cols = [
    'STORUNST', 'FROR_CD', 'KOFTR_GROU', 'KOFTR_NM',
    'HEIGHT', 'HEIGHT_NM',
    'LDMARK_STN', 'MAP_LABEL',
    '갱신년도', 'ETC_PCMTT',
    'FRTP_NM', 'DMCLS_CD', 'AGCLS_CD', 'DNST_CD',
    'DMCLS_NM', 'AGCLS_NM', 'DNST_NM',
    'Shape_Leng', 'Shape_Area'
]

In [ ]:
gdf_cleaned = gdf.drop(columns=drop_cols).copy()

print(gdf_cleaned.columns.tolist())

In [ ]:
gdf_cleaned.head()

In [ ]:
gdf_cleaned.to_file("../data/processed/서울_임상도_전처리.gpkg", driver="GPKG")

### 대한민국 행정구역 데이터 전처리

In [27]:
gdf = gpd.read_file("../data/raw/sig_20230729/sig.shp", encoding="euc-kr")

In [28]:
# 1. 좌표계 설정
gdf.set_crs(epsg=5179, inplace=True)

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,geometry
0,11110,Jongno-gu,종로구,"POLYGON ((956615.453 1953567.199, 956621.579 1..."
1,11140,Jung-gu,중구,"POLYGON ((957890.386 1952616.746, 957909.908 1..."
2,11170,Yongsan-gu,용산구,"POLYGON ((953115.761 1950834.084, 953114.206 1..."
3,11200,Seongdong-gu,성동구,"POLYGON ((959681.109 1952649.605, 959842.412 1..."
4,11215,Gwangjin-gu,광진구,"POLYGON ((964825.058 1952633.25, 964875.565 19..."
...,...,...,...,...
245,51790,Hwacheon-gun,화천군,"POLYGON ((1027779.614 2032000.017, 1027951.841..."
246,51800,Yanggu-gun,양구군,"MULTIPOLYGON (((1046391.747 2032451.649, 10463..."
247,51810,Inje-gun,인제군,"POLYGON ((1041948.853 1995343.843, 1042054.076..."
248,51820,Goseong-gun,고성군,"MULTIPOLYGON (((1091705.056 2034023.203, 10917..."


In [29]:
# 필요한 컬럼만 남기기
gdf = gdf[["SIG_CD", "SIG_KOR_NM", "geometry"]].copy()

# 컬럼명 변경
gdf = gdf.rename(columns={"SIG_CD": "시군구코드", "SIG_KOR_NM": "시군구명"})

# geometry 유효성 검사
gdf = gdf[gdf.is_valid]

# 면적 0 제거 (혹시라도 있을 경우)
gdf = gdf[gdf.geometry.area > 0]

# 면적 컬럼 추가
gdf["면적"] = gdf.geometry.area

# 시도 코드 추출 (선택)
gdf["시도코드"] = gdf["시군구코드"].str[:2]

In [35]:
sido_map = {
    '11': '서울특별시',
    '26': '부산광역시',
    '27': '대구광역시',
    '28': '인천광역시',
    '29': '광주광역시',
    '30': '대전광역시',
    '31': '울산광역시',
    '36': '세종특별자치시',
    '41': '경기도',
    '42': '강원도',
    '43': '충청북도',
    '44': '충청남도',
    '45': '전라북도',
    '46': '전라남도',
    '47': '경상북도',
    '48': '경상남도',
    '49': '제주도',
    '50': '제주특별자치도',
    '51': '기타'
}

gdf["시도명"] = gdf["시도코드"].map(sido_map)

In [36]:
gdf.head()

,시군구코드,시군구명,geometry,면적,시도코드,시도명
0,11110,종로구,"POLYGON ((956615.453 1953567.199, 956621.579 1...",2.397161e+07,11,서울특별시
1,11140,중구,"POLYGON ((957890.386 1952616.746, 957909.908 1...",9.962768e+06,11,서울특별시
2,11170,용산구,"POLYGON ((953115.761 1950834.084, 953114.206 1...",2.189756e+07,11,서울특별시
3,11200,성동구,"POLYGON ((959681.109 1952649.605, 959842.412 1...",1.680078e+07,11,서울특별시
4,11215,광진구,"POLYGON ((964825.058 1952633.25, 964875.565 19...",1.702881e+07,11,서울특별시


In [37]:
gdf.to_file("../data/processed/대한민국_시군구.gpkg", driver="GPKG")

### 산불 데이터 전처리

In [101]:
import pandas as pd

In [102]:
df = pd.read_csv("../data/raw/산림청_산불상황관제시스템_산불통계데이터_20241016.csv", encoding="cp949")

In [103]:
df

,발생일시_년,발생일시_월,발생일시_일,발생일시_시간,발생일시_요일,진화종료시간_년,진화종료시간_월,진화종료시간_일,진화종료시간_시간,발생장소_관서,발생장소_시도,발생장소_시군구,발생장소_읍면,발생장소_동리,발생원인_구분,발생원인_세부원인,발생원인_기타,피해면적_합계
0,2024,9,29,15:41,일,2024,9,30,16:30,전북,전북,남원,산동,부절,기,작업장실화,산업현장실화,0.31
1,2024,9,10,15:55,화,2024,9,10,18:00,경남,경남,밀양,NaN,내이,기,기타(직접입력),성묘객실화(벌집소각),0.10
2,2024,9,10,14:35,화,2024,9,10,17:52,충남,충남,부여,규암,수목,기,기타(직접입력),원인미상,0.03
3,2024,9,10,14:24,화,2024,9,10,22:00,경북,경북,상주,화동,신촌,기,기타(직접입력),조사중,1.00
4,2024,9,5,13:51,목,2024,9,5,16:10,경북,경북,안동,녹전,매정,NaN,기타(직접입력),농산폐기물소각,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,2022,1,3,12:36,월,2022,1,3,15:35,대구,대구,달성,옥포,반송,기,기타(직접입력),화목난로비화,0.08
1566,2022,1,3,12:02,월,2022,1,3,14:56,강원,강원,인제,남,수산,기,기타(직접입력),화목보일러재투기,0.80
1567,2022,1,2,14:29,일,2022,1,2,16:50,전남,전남,보성,조성,귀산,담,담뱃불실화,담뱃불실화,0.10
1568,2022,1,2,08:18,일,2022,1,2,12:50,경북,경북,군위,부계,신화,NaN,주택화재비화,축사전기누전,0.20


In [104]:
df.columns

Index(['발생일시_년', '발생일시_월', '발생일시_일', '발생일시_시간', '발생일시_요일', '진화종료시간_년',
       '진화종료시간_월', '진화종료시간_일', '진화종료시간_시간', '발생장소_관서', '발생장소_시도', '발생장소_시군구',
       '발생장소_읍면', '발생장소_동리', '발생원인_구분', '발생원인_세부원인', '발생원인_기타', '피해면적_합계'],
      dtype='object')

In [105]:
# 1. 발생일시 및 진화종료시각 생성
df["발생일시"] = pd.to_datetime(
    df[["발생일시_년", "발생일시_월", "발생일시_일"]].astype(str).agg("-".join, axis=1) + " " + df["발생일시_시간"],
    errors="coerce"
)

df["진화종료시각"] = pd.to_datetime(
    df[["진화종료시간_년", "진화종료시간_월", "진화종료시간_일"]].astype(str).agg("-".join, axis=1) + " " + df["진화종료시간_시간"],
    errors="coerce"
)

In [106]:
# 2. 행정구역 문자열 생성
# df["행정구역"] = df[
#     ["발생장소_시도", "발생장소_시군구", "발생장소_읍면", "발생장소_동리"]
# ].fillna("").agg(" ".join, axis=1).str.strip()

# 시도 + 시군구까지만 결합
# df["행정구역"] = df[["발생장소_시도", "발생장소_시군구"]].fillna("").agg(" ".join, axis=1).str.strip()

In [107]:
# 3. 원인 통합
df["원인"] = df[
    ["발생원인_구분", "발생원인_세부원인", "발생원인_기타"]
].fillna("").agg(" / ".join, axis=1).str.strip(" /")

In [108]:
# 4. 피해면적 변환 (헥타르 → 제곱미터)
df["피해면적_m2"] = df["피해면적_합계"] * 10000

In [109]:
# 5. 파생 컬럼 추가 (월, 계절)
df["발생월"] = df["발생일시"].dt.month
df["계절"] = df["발생월"].map({
    12: "겨울", 1: "겨울", 2: "겨울",
    3: "봄", 4: "봄", 5: "봄",
    6: "여름", 7: "여름", 8: "여름",
    9: "가을", 10: "가을", 11: "가을"
})

In [110]:
# 시간대 분류
def get_time_period(hour):
    if pd.isna(hour):
        return None
    if hour < 6:
        return "새벽"
    elif hour < 12:
        return "오전"
    elif hour < 18:
        return "오후"
    else:
        return "밤"

df["발생시간대"] = df["발생일시"].dt.hour.apply(get_time_period)

In [111]:
df["진화시간_분"] = (df["진화종료시각"] - df["발생일시"]).dt.total_seconds() / 60

In [112]:
df.rename(columns={"발생장소_시도":"시도", "발생장소_시군구":"시군구"}, inplace=True)

In [113]:
# 유지할 컬럼만 명시
# keep_cols = [
#     "발생일시", "진화종료시각", "진화시간_분", "행정구역", "시도", "시군구", "원인",
#     "피해면적_m2", "계절", "발생시간대", "발생월"
# ]

keep_cols = [
    "발생일시", "진화종료시각", "진화시간_분", "시도", "시군구", "원인",
    "피해면적_m2", "계절", "발생시간대", "발생월"
]

# 나머지 컬럼 삭제
df = df[keep_cols].copy()

# 확인
print(df.columns)

Index(['발생일시', '진화종료시각', '진화시간_분', '시도', '시군구', '원인', '피해면적_m2', '계절', '발생시간대',
       '발생월'],
      dtype='object')


In [114]:
df

,발생일시,진화종료시각,진화시간_분,시도,시군구,원인,피해면적_m2,계절,발생시간대,발생월
0,2024-09-29 15:41:00,2024-09-30 16:30:00,1489.0,전북,남원,기 / 작업장실화 / 산업현장실화,3100.0,가을,오후,9
1,2024-09-10 15:55:00,2024-09-10 18:00:00,125.0,경남,밀양,기 / 기타(직접입력) / 성묘객실화(벌집소각),1000.0,가을,오후,9
2,2024-09-10 14:35:00,2024-09-10 17:52:00,197.0,충남,부여,기 / 기타(직접입력) / 원인미상,300.0,가을,오후,9
3,2024-09-10 14:24:00,2024-09-10 22:00:00,456.0,경북,상주,기 / 기타(직접입력) / 조사중,10000.0,가을,오후,9
4,2024-09-05 13:51:00,2024-09-05 16:10:00,139.0,경북,안동,기타(직접입력) / 농산폐기물소각,500.0,가을,오후,9
...,...,...,...,...,...,...,...,...,...,...
1565,2022-01-03 12:36:00,2022-01-03 15:35:00,179.0,대구,달성,기 / 기타(직접입력) / 화목난로비화,800.0,겨울,오후,1
1566,2022-01-03 12:02:00,2022-01-03 14:56:00,174.0,강원,인제,기 / 기타(직접입력) / 화목보일러재투기,8000.0,겨울,오후,1
1567,2022-01-02 14:29:00,2022-01-02 16:50:00,141.0,전남,보성,담 / 담뱃불실화 / 담뱃불실화,1000.0,겨울,오후,1
1568,2022-01-02 08:18:00,2022-01-02 12:50:00,272.0,경북,군위,주택화재비화 / 축사전기누전,2000.0,겨울,오전,1


In [115]:
sido_map = {
    "서울": "서울특별시", "부산": "부산광역시", "대구": "대구광역시",
    "인천": "인천광역시", "광주": "광주광역시", "대전": "대전광역시",
    "울산": "울산광역시", "세종": "세종특별자치시",
    "경기": "경기도", "강원": "강원도", "충북": "충청북도", "충남": "충청남도",
    "전북": "전라북도", "전남": "전라남도", "경북": "경상북도", "경남": "경상남도",
    "제주": "제주특별자치도"
}
df["시도"] = df["시도"].map(sido_map).fillna(df["시도"])

In [116]:
def format_sigungu(sido, sigungu):
    if pd.isna(sigungu):
        return ""
    sigungu = str(sigungu).strip()
    sido = str(sido).strip()

    parts = sigungu.split()
    if len(parts) == 2:
        return parts[0] + "시 " + parts[1] + "구"
    elif sido.endswith("특별시") or sido.endswith("광역시"):
        return sigungu + "구" if not sigungu.endswith("구") else sigungu
    elif sigungu.endswith(("시", "군", "구")):
        return sigungu
    else:
        return sigungu + "시"

In [117]:
df["시군구"] = df.apply(lambda row: format_sigungu(row["시도"], row["시군구"]), axis=1)
df["행정구역"] = df["시도"] + " " + df["시군구"]

In [120]:
df

,발생일시,진화종료시각,진화시간_분,시도,시군구,원인,피해면적_m2,계절,발생시간대,발생월,행정구역
0,2024-09-29 15:41:00,2024-09-30 16:30:00,1489.0,전라북도,남원시,기 / 작업장실화 / 산업현장실화,3100.0,가을,오후,9,전라북도 남원시
1,2024-09-10 15:55:00,2024-09-10 18:00:00,125.0,경상남도,밀양시,기 / 기타(직접입력) / 성묘객실화(벌집소각),1000.0,가을,오후,9,경상남도 밀양시
2,2024-09-10 14:35:00,2024-09-10 17:52:00,197.0,충청남도,부여시,기 / 기타(직접입력) / 원인미상,300.0,가을,오후,9,충청남도 부여시
3,2024-09-10 14:24:00,2024-09-10 22:00:00,456.0,경상북도,상주시,기 / 기타(직접입력) / 조사중,10000.0,가을,오후,9,경상북도 상주시
4,2024-09-05 13:51:00,2024-09-05 16:10:00,139.0,경상북도,안동시,기타(직접입력) / 농산폐기물소각,500.0,가을,오후,9,경상북도 안동시
...,...,...,...,...,...,...,...,...,...,...,...
1565,2022-01-03 12:36:00,2022-01-03 15:35:00,179.0,대구광역시,달성구,기 / 기타(직접입력) / 화목난로비화,800.0,겨울,오후,1,대구광역시 달성구
1566,2022-01-03 12:02:00,2022-01-03 14:56:00,174.0,강원도,인제시,기 / 기타(직접입력) / 화목보일러재투기,8000.0,겨울,오후,1,강원도 인제시
1567,2022-01-02 14:29:00,2022-01-02 16:50:00,141.0,전라남도,보성시,담 / 담뱃불실화 / 담뱃불실화,1000.0,겨울,오후,1,전라남도 보성시
1568,2022-01-02 08:18:00,2022-01-02 12:50:00,272.0,경상북도,군위시,주택화재비화 / 축사전기누전,2000.0,겨울,오전,1,경상북도 군위시


In [121]:
df.to_csv("../data/processed/산불데이터_전처리.csv", index=False, encoding="utf-8-sig")